#### Wildfire Severity Prediction using Raster Tools and SciKit-Learn

## 1. Data Collection

## 2. Building the Dataset

##### Once the data has been collected, the dataset can be built.  The files must be arranged in specific folders for the script to work.  The structure is as follows:

> <table><th>Feature</th><th>Folder</th></tr>
<tr><td>Elevation Data</td><td>data/terrain</td></tr>
<tr><td>GridMet Climate Data</td><td>data/FeatureData/gridmet</td></tr>
<tr><td>DayMet Climate Data</td><td>data/FeatureData/daymet</td></tr>
<tr><td>AdaptWest Climate Data</td><td>data/FeatureData/adaptwest</td></tr>
<tr><td>Landfire Fuel Data</td><td>data/FeatureData/landfire</td></tr>
<tr><td>Biomass Data</td><td>data/FeatureData/biomass</td></tr>
<tr><td>MTBS Data</td><td>data/MTBS_Data</td></tr>

## 3. Model Training

## 4. Model Evaluation